## Cleaning flights.csv

**NOTE::** This is only one approach to cleaning up this dataset.  Notice that an assumptions I've made are well documented.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pylab as plt
%matplotlib inline

### Loading data

In [2]:
data_df = pd.read_csv('assign_wk2/flights.csv', low_memory=False)

<div class="alert alert-block alert-info">
<b>pd.read_csv low_memory argument: Why did I include this?</b>  <br>
There are a couple of columns within the dataset that have a mixed data type.  I was able to see this when I visually inspected the data file in Sublime first. If you didn't do this, Pandas would have presented a warning informing you of this.<br>
<b>Absence of dropna(): Yes I could have done that when loading the data.</b>  <br>
However, if I did that when I loaded the dataset, I would have lost several of the columns that I need for this analysis. So, we will need to clean things up after a bit of analysis.
</div>

In [3]:
data_df.head(10)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5,2015,1,1,4,DL,806,N3730B,SFO,MSP,25,...,610.0,8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
6,2015,1,1,4,NK,612,N635NK,LAS,MSP,25,...,509.0,-17.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
7,2015,1,1,4,US,2013,N584UW,LAX,CLT,30,...,753.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
8,2015,1,1,4,AA,1112,N3LAAA,SFO,DFW,30,...,532.0,-13.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
9,2015,1,1,4,DL,1173,N826DN,LAS,ATL,30,...,656.0,-15.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

<div class="alert alert-block alert-info">
<b>Notice anything missing? Where did the non-null attribute information go?</b>  <br>
Since our dataset is so large, that information is excluded from this view.  So we will have test for it outside of info() function.
</div>

In [5]:
# count the number of NaN in each column of the dataset
data_df.isnull().sum()

YEAR                         0
MONTH                        0
DAY                          0
DAY_OF_WEEK                  0
AIRLINE                      0
FLIGHT_NUMBER                0
TAIL_NUMBER              14721
ORIGIN_AIRPORT               0
DESTINATION_AIRPORT          0
SCHEDULED_DEPARTURE          0
DEPARTURE_TIME           86153
DEPARTURE_DELAY          86153
TAXI_OUT                 89047
WHEELS_OFF               89047
SCHEDULED_TIME               6
ELAPSED_TIME            105071
AIR_TIME                105071
DISTANCE                     0
WHEELS_ON                92513
TAXI_IN                  92513
SCHEDULED_ARRIVAL            0
ARRIVAL_TIME             92513
ARRIVAL_DELAY           105071
DIVERTED                     0
CANCELLED                    0
CANCELLATION_REASON    5729195
AIR_SYSTEM_DELAY       4755640
SECURITY_DELAY         4755640
AIRLINE_DELAY          4755640
LATE_AIRCRAFT_DELAY    4755640
WEATHER_DELAY          4755640
dtype: int64

In [6]:
# converting all the column names to lowercase() - personal preference
data_df.columns = data_df.columns.str.lower()

### Imputing the arrival_delay column

We can start to determine which columns we don't feel will support our analysis. Based on the fact that our dataset has 5.8+ million rows, we can see that the last six columns are missing over 50% of their data.  So, I'm going to remove those columns. 

Additionally, our analysis is centered around arrival_delay and the originating airport, so we want to keep columns that might support that analysis. Columns that I'm going to keep at this point are:
   - date information: year, month, day, day_of_week
   - info to uniquely identify the flight: airline, flight_number, origin_airport,destination_airport
   - departure info: departure_time, departure_delay, scheduled_departure, scheduled_time, elapse_time
   - arrival info: scheduled_arrival, arrival_time, arrival_delay
   - canceled/diverted flight info: diverted, cancelled

In [7]:
drop_cols = ['tail_number','taxi_out','wheels_off','air_time','distance','wheels_on','taxi_in','cancellation_reason','air_system_delay','security_delay','airline_delay','late_aircraft_delay','weather_delay']

In [8]:
data_df.drop(drop_cols,axis=1,inplace=True)

In [9]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 18 columns):
 #   Column               Dtype  
---  ------               -----  
 0   year                 int64  
 1   month                int64  
 2   day                  int64  
 3   day_of_week          int64  
 4   airline              object 
 5   flight_number        int64  
 6   origin_airport       object 
 7   destination_airport  object 
 8   scheduled_departure  int64  
 9   departure_time       float64
 10  departure_delay      float64
 11  scheduled_time       float64
 12  elapsed_time         float64
 13  scheduled_arrival    int64  
 14  arrival_time         float64
 15  arrival_delay        float64
 16  diverted             int64  
 17  cancelled            int64  
dtypes: float64(6), int64(9), object(3)
memory usage: 799.1+ MB


Now we can start address filling in the missing arrival_delay values. <br>

I'm going to define an arrival delay to be based on the scheduled_arrival - arrival_time.

In [10]:
data_df.head(10)

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled
0,2015,1,1,4,AS,98,ANC,SEA,5,2354.0,-11.0,205.0,194.0,430,408.0,-22.0,0,0
1,2015,1,1,4,AA,2336,LAX,PBI,10,2.0,-8.0,280.0,279.0,750,741.0,-9.0,0,0
2,2015,1,1,4,US,840,SFO,CLT,20,18.0,-2.0,286.0,293.0,806,811.0,5.0,0,0
3,2015,1,1,4,AA,258,LAX,MIA,20,15.0,-5.0,285.0,281.0,805,756.0,-9.0,0,0
4,2015,1,1,4,AS,135,SEA,ANC,25,24.0,-1.0,235.0,215.0,320,259.0,-21.0,0,0
5,2015,1,1,4,DL,806,SFO,MSP,25,20.0,-5.0,217.0,230.0,602,610.0,8.0,0,0
6,2015,1,1,4,NK,612,LAS,MSP,25,19.0,-6.0,181.0,170.0,526,509.0,-17.0,0,0
7,2015,1,1,4,US,2013,LAX,CLT,30,44.0,14.0,273.0,249.0,803,753.0,-10.0,0,0
8,2015,1,1,4,AA,1112,SFO,DFW,30,19.0,-11.0,195.0,193.0,545,532.0,-13.0,0,0
9,2015,1,1,4,DL,1173,LAS,ATL,30,33.0,3.0,221.0,203.0,711,656.0,-15.0,0,0


Now let's take a closer look at only rows the are missing an arrival_delay.

In [11]:
data_df[data_df.arrival_delay.isnull()].head(10)

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled
32,2015,1,1,4,AS,136,ANC,SEA,135,NaN,NaN,205.0,NaN,600,NaN,NaN,0,1
42,2015,1,1,4,AA,2459,PHX,DFW,200,NaN,NaN,120.0,NaN,500,NaN,NaN,0,1
68,2015,1,1,4,OO,5254,MAF,IAH,510,NaN,NaN,87.0,NaN,637,NaN,NaN,0,1
82,2015,1,1,4,MQ,2859,SGF,DFW,525,NaN,NaN,95.0,NaN,700,NaN,NaN,0,1
90,2015,1,1,4,OO,5460,RDD,SFO,530,NaN,NaN,90.0,NaN,700,NaN,NaN,0,1
128,2015,1,1,4,MQ,2926,CHS,DFW,545,NaN,NaN,190.0,NaN,755,NaN,NaN,0,1
131,2015,1,1,4,OO,6457,SMX,LAX,545,NaN,NaN,66.0,NaN,651,NaN,NaN,0,1
147,2015,1,1,4,MQ,3534,ABI,DFW,550,NaN,NaN,55.0,NaN,645,NaN,NaN,0,1
166,2015,1,1,4,MQ,3161,XNA,DFW,555,NaN,NaN,75.0,NaN,710,NaN,NaN,0,1
206,2015,1,1,4,AA,175,DCA,DFW,600,NaN,NaN,215.0,NaN,835,NaN,NaN,0,1


<div class="alert alert-block alert-info">
<b>Index numbers: Why are the index number to the far left non-sequential at this point?</b>  <br>
We asked to only see the rows of data that are missing an arrival_delay value. The index number to the far left is showing the row number (aka position) in the overall dataframe.
</div>

Very interesting! If we scroll to the right, we see that a majority of the flights missing an arrival_delay value were canceled. I'm going to contend that a canceled flight can't be delayed and falls outside our intended analysis. So, I'm going to drop rows where the flight was canceled.

In [12]:
data_df.drop(data_df[data_df.cancelled == 1].index, inplace=True)

We can use value_counts to verify that we only have rows for flights that actually occurred (cancelled = 0).

In [13]:
data_df.cancelled.value_counts()

0    5729195
Name: cancelled, dtype: int64

So far so good, time to see how many missing arrival_delay values we have at this point.

In [14]:
data_df[data_df.arrival_delay.isnull()].shape

(15187, 18)

In [15]:
data_df.shape

(5729195, 18)

Wow!!! That dropped the number of rows with a missing arrival_delay value from over 105K to around 15K. Also, the overall size of our dataset wasn't minimally reduced in sized. Things are looking good at this point!

In [16]:
data_df[data_df.arrival_delay.isnull()].head(10)

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled
724,2015,1,1,4,DL,716,OMA,ATL,645,637.0,-8.0,144.0,NaN,1009,1451.0,NaN,1,0
1455,2015,1,1,4,OO,5237,MKE,IAH,745,742.0,-3.0,176.0,NaN,1041,1505.0,NaN,1,0
2218,2015,1,1,4,WN,1966,ATL,JAX,845,922.0,37.0,75.0,NaN,1000,1219.0,NaN,1,0
2526,2015,1,1,4,EV,4555,IAH,HRL,902,859.0,-3.0,75.0,NaN,1017,1319.0,NaN,1,0
2866,2015,1,1,4,WN,1081,MDW,OKC,930,926.0,-4.0,125.0,NaN,1135,1343.0,NaN,1,0
4225,2015,1,1,4,AA,1279,OMA,DFW,1100,1058.0,-2.0,120.0,NaN,1300,1838.0,NaN,1,0
4394,2015,1,1,4,OO,4506,SLC,SUN,1110,1108.0,-2.0,75.0,NaN,1225,1353.0,NaN,1,0
4574,2015,1,1,4,EV,6171,DEN,GUC,1122,1120.0,-2.0,56.0,NaN,1218,1635.0,NaN,1,0
4636,2015,1,1,4,EV,4169,IAH,ABQ,1125,1122.0,-3.0,138.0,NaN,1243,1630.0,NaN,1,0
4639,2015,1,1,4,EV,4654,IAH,HRL,1125,1206.0,41.0,75.0,NaN,1240,NaN,NaN,1,0


Well, it's time to just start making some assumptions and documenting our process. Here is my approach to filling in the missing arrival_delay values.
   1. if we can calculate the arrival_delay value based on scheduled_arrival and arrival_time
   2. if a flight leaves early or on-time, it will arrive early the same number of minutes
   3. if a flight leaves late, 15% or less than the flight duration, it will make that time up in the air and arrive on time
   4. if a fligt leaves late, more than 15% of the flight duration, it will be late the amount of time they left late - 15% of the flight duration.

I'm going to create a UDF and then use a combination of apply() and lambda to fill in the missing arrival_delay values.

In [17]:
def fill_missing_delay(row):
    delay = np.NaN
    
    if np.isnan(row.arrival_delay):
        if ~np.isnan(row.scheduled_arrival) and ~np.isnan(row.arrival_time):
            delay = row.scheduled_arrival - row.arrival_time
        elif row.departure_delay <=  0:
            delay = np.negative(row.departure_delay)
        elif row.departure_delay <=  (0.15 * row.scheduled_time):
            delay = float(0)
        else:
            delay = np.negative(row.departure_delay - (0.15 * row.scheduled_time))
    else:
        delay = row.arrival_delay
        
    return delay

<div class="alert alert-block alert-info">
<b>Special Character: What are the '~' used for above?</b>  <br>
'~': is a way to negate a statement. So ~ np.isnan(xxx) means that we are testing that xxx does not equal NaN
</div>

In [18]:
data_df.arrival_delay = data_df.apply(lambda x: fill_missing_delay(x), axis = 1)

In [19]:
data_df[data_df.arrival_delay.isnull()].head(10)

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled
1720237,2015,4,20,1,NK,174,FLL,LGA,1602,1857.0,175.0,NaN,NaN,1900,NaN,NaN,1,0


In [20]:
data_df[data_df.arrival_delay.isnull()].shape

(1, 18)

Awesome!!! We are down to 1 flight that needs a value. It looks like the issue with this row is the missing value for scheduled_time. Let's see if there are other flights that go between the origination and destination airports that we can use as a baseline.

In [21]:
data_df[(data_df.origin_airport == 'FLL') \
        & (data_df.destination_airport == 'LGA') \
        & (data_df.airline == 'NK')]

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled
1068,2015,1,1,4,NK,604,FLL,LGA,712,709.0,-3.0,168.0,190.0,1000,1019.0,19.0,0,0
4740,2015,1,1,4,NK,180,FLL,LGA,1130,1232.0,62.0,171.0,167.0,1421,1519.0,58.0,0,0
9174,2015,1,1,4,NK,174,FLL,LGA,1610,1637.0,27.0,170.0,161.0,1900,1918.0,18.0,0,0
12644,2015,1,1,4,NK,710,FLL,LGA,2009,2010.0,1.0,167.0,158.0,2256,2248.0,-8.0,0,0
15897,2015,1,2,5,NK,604,FLL,LGA,712,739.0,27.0,168.0,163.0,1000,1022.0,22.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5804263,2015,12,30,3,NK,710,FLL,LGA,2011,2043.0,32.0,168.0,148.0,2259,2311.0,12.0,0,0
5807641,2015,12,31,4,NK,604,FLL,LGA,710,709.0,-1.0,175.0,152.0,1005,941.0,-24.0,0,0
5811544,2015,12,31,4,NK,180,FLL,LGA,1130,1125.0,-5.0,170.0,161.0,1420,1406.0,-14.0,0,0
5815589,2015,12,31,4,NK,174,FLL,LGA,1606,1606.0,0.0,174.0,162.0,1900,1848.0,-12.0,0,0


<div class="alert alert-block alert-info">
<b>Another Special Character: How about the '/'? What are they for?</b>  <br>
'\': is a line continuation marker and simply means that the code continues on the following line. <br>
</div>

Alright, I'm going to assume that flight duration for our 1 row above is the mean of all the other flights going between FLL and LGA and fill in the missing scheduled_time for this 1 row. <br>
<br>
I'm going to use a couple of intermediate variables to condense the code a bit.

In [22]:
avg_duration = round(data_df[(data_df.origin_airport == 'FLL') \
                             & (data_df.destination_airport == 'LGA') \
                             & (data_df.airline == 'NK')].scheduled_time.mean())
row_delay_departure = data_df[data_df.arrival_delay.isnull()].departure_delay.sum()                 

In [23]:
data_df.arrival_delay.fillna(float(np.negative(row_delay_departure - (0.15 * avg_duration))), inplace=True)

In [24]:
data_df[data_df.arrival_delay.isnull()].shape

(0, 18)

Hooray! We have managed to cleanup all of the arrival_delay missing values. Time proceed with the analysis.

#### Weird originating airport codes
Since I look at my data in a text editior prior to loading it, I noticed something interesting with the range of values in the originating_airport column.

In [25]:
data_df.origin_airport.unique()

array(['ANC', 'LAX', 'SFO', 'SEA', 'LAS', 'DEN', 'SLC', 'PDX', 'FAI',
       'MSP', 'PHX', 'SJU', 'PBG', 'IAG', 'PSE', 'BQN', 'ORD', 'GEG',
       'HNL', 'ONT', 'MCO', 'BOS', 'HIB', 'ABR', 'DFW', 'MKE', 'IAH',
       'BNA', 'BRO', 'VPS', 'BOI', 'BJI', 'PHL', 'SBN', 'EUG', 'IAD',
       'BUF', 'PWM', 'JFK', 'CRP', 'PIA', 'FAT', 'SMF', 'AUS', 'MCI',
       'ATL', 'JAX', 'MFR', 'IDA', 'MSN', 'DCA', 'SAT', 'CHS', 'SBA',
       'IND', 'CLE', 'GSP', 'BDL', 'RIC', 'BFL', 'OMA', 'RDM', 'FLL',
       'CID', 'TPA', 'SYR', 'ROC', 'TYR', 'LAN', 'GSO', 'EWR', 'PBI',
       'RSW', 'OAK', 'PVD', 'RNO', 'PIT', 'ABQ', 'MIA', 'BWI', 'TUL',
       'LGA', 'LIT', 'MSY', 'OKC', 'ATW', 'PNS', 'MEM', 'TYS', 'MHT',
       'SAV', 'CLT', 'GRB', 'ABE', 'JAN', 'OAJ', 'FAR', 'ERI', 'LEX',
       'CWA', 'TTN', 'RDU', 'CVG', 'BHM', 'ACY', 'DTW', 'RAP', 'TUS',
       'EAU', 'DLH', 'FSD', 'INL', 'SPI', 'CLD', 'COD', 'CMH', 'PSC',
       'CPR', 'ACV', 'DAL', 'PAH', 'MRY', 'ESC', 'ISN', 'PSP', 'CAE',
       'STL', 'BTV',

Very intersting, why do we have airports with a numerical name.  This took some research and the FAA reports that these a smaller community/regional airports. I'm going to limit my analysis to major airports and remove these from the dataset. To do this, I'm going to create a temporary column that contains the length of the originating airport name. Based on visual inspection, I should only have 3 or 5 in this column. Then I'll drop all the rows with a length of 5.

In [26]:
data_df['name_len'] = 0
data_df.name_len = data_df.origin_airport.apply(lambda x: len(str(x)))

In [27]:
data_df.name_len.value_counts()

3    5245484
5     483711
Name: name_len, dtype: int64

In [28]:
data_df.drop(data_df[data_df.name_len == 5].index, inplace=True)

Double check our changes!

In [29]:
data_df.origin_airport.unique()

array(['ANC', 'LAX', 'SFO', 'SEA', 'LAS', 'DEN', 'SLC', 'PDX', 'FAI',
       'MSP', 'PHX', 'SJU', 'PBG', 'IAG', 'PSE', 'BQN', 'ORD', 'GEG',
       'HNL', 'ONT', 'MCO', 'BOS', 'HIB', 'ABR', 'DFW', 'MKE', 'IAH',
       'BNA', 'BRO', 'VPS', 'BOI', 'BJI', 'PHL', 'SBN', 'EUG', 'IAD',
       'BUF', 'PWM', 'JFK', 'CRP', 'PIA', 'FAT', 'SMF', 'AUS', 'MCI',
       'ATL', 'JAX', 'MFR', 'IDA', 'MSN', 'DCA', 'SAT', 'CHS', 'SBA',
       'IND', 'CLE', 'GSP', 'BDL', 'RIC', 'BFL', 'OMA', 'RDM', 'FLL',
       'CID', 'TPA', 'SYR', 'ROC', 'TYR', 'LAN', 'GSO', 'EWR', 'PBI',
       'RSW', 'OAK', 'PVD', 'RNO', 'PIT', 'ABQ', 'MIA', 'BWI', 'TUL',
       'LGA', 'LIT', 'MSY', 'OKC', 'ATW', 'PNS', 'MEM', 'TYS', 'MHT',
       'SAV', 'CLT', 'GRB', 'ABE', 'JAN', 'OAJ', 'FAR', 'ERI', 'LEX',
       'CWA', 'TTN', 'RDU', 'CVG', 'BHM', 'ACY', 'DTW', 'RAP', 'TUS',
       'EAU', 'DLH', 'FSD', 'INL', 'SPI', 'CLD', 'COD', 'CMH', 'PSC',
       'CPR', 'ACV', 'DAL', 'PAH', 'MRY', 'ESC', 'ISN', 'PSP', 'CAE',
       'STL', 'BTV',

No need to keep the name_len column at this point.

In [30]:
data_df.drop(['name_len'],axis=1,inplace=True)

In [31]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5245484 entries, 0 to 5819078
Data columns (total 18 columns):
 #   Column               Dtype  
---  ------               -----  
 0   year                 int64  
 1   month                int64  
 2   day                  int64  
 3   day_of_week          int64  
 4   airline              object 
 5   flight_number        int64  
 6   origin_airport       object 
 7   destination_airport  object 
 8   scheduled_departure  int64  
 9   departure_time       float64
 10  departure_delay      float64
 11  scheduled_time       float64
 12  elapsed_time         float64
 13  scheduled_arrival    int64  
 14  arrival_time         float64
 15  arrival_delay        float64
 16  diverted             int64  
 17  cancelled            int64  
dtypes: float64(6), int64(9), object(3)
memory usage: 760.4+ MB


In [32]:
data_df.shape

(5245484, 18)

Write the cleaned version of the dataset to a csv

In [35]:
data_df.to_csv('assign_wk2/flights_clean.csv',index=False)